In [190]:
%load_ext autoreload
%autoreload 2
import glob
import numpy as np
import matplotlib.pyplot as plt

import hyperspy.api as hs
from dask.distributed import Client

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [120]:
save_path='/home/m3learning/Northwestern/Northwestern_link/Northwestern_Multimodal'
eels_ll_list = glob.glob(f'{save_path}/TRI*/eels*/EELS LL SI.dm4')
eels_hl_list = glob.glob(f'{save_path}/TRI*/eels*/EELS HL SI.dm4')
eels_ll_list.sort(),eels_hl_list.sort();


In [121]:
lpath,hpath = eels_ll_list[:5], eels_hl_list[:5]
ll_spectrum = hs.load(lpath[:5],lazy=True)
hl_spectrum = hs.load(hpath[:5],lazy=True)

In [122]:
ll_spectrum,hl_spectrum

([<LazyEELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1024)>,
  <LazyEELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1024)>,
  <LazyEELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1024)>,
  <LazyEELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1024)>,
  <LazyEELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1024)>],
 [<LazyEELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1024)>,
  <LazyEELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1024)>,
  <LazyEELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1024)>,
  <LazyEELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1024)>,
  <LazyEELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1024)>])

In [123]:
# Try 0-peak alignment in parallel: over 1 min
# from concurrent.futures import ThreadPoolExecutor, as_completed

# def align_spectrum(ll,hl):
#     try:
#         ll.compute()
#         hl.compute()
#     except: pass
#     ll.align_zero_loss_peak(calibrate=True, also_align=[hl])
    
# with ThreadPoolExecutor() as executor:
#     futures = [executor.submit(align_spectrum, ll_spectrum[i], hl_spectrum[i]) for i in range(len(ll_spectrum))]
#     for future in as_completed(futures):
#         ll, hl = future.result()
#         print(f'Aligned spectra: {ll}, {hl}')


In [124]:
for i in range(len(ll_spectrum)):
    try:
        ll_spectrum[i].compute()
        hl_spectrum[i].compute()
    except: pass
    ll_spectrum[i].align_zero_loss_peak(calibrate=True, also_align=[hl_spectrum[i]], expand=False)

[########################################] | 100% Completed | 103.63 ms
[########################################] | 100% Completed | 103.35 ms
Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-2.8
std:	0.143

min:	-3.3
Q1:	-3
median:	-2.7
Q3:	-2.7
max:	-2.4
[########################################] | 100% Completed | 2.73 ss
[########################################] | 100% Completed | 2.73 ss
[########################################] | 100% Completed | 3.28 ss
[########################################] | 100% Completed | 2.35 sms
[########################################] | 100% Completed | 2.34 sms
[########################################] | 100% Completed | 103.07 ms
[########################################] | 100% Completed | 104.17 ms
Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-4.57
std:	0.203

min:	-5.1
Q1:	-4.8
median:	-4.5
Q3:	-4.5
max:	-4.2
[########

In [125]:
ll_spectrum,hl_spectrum

([<EELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1017)>,
  <EELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1018)>,
  <EELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1017)>,
  <EELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1015)>,
  <EELSSpectrum, title: EELS LL SI, dimensions: (140, 140|1015)>],
 [<EELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1017)>,
  <EELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1018)>,
  <EELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1017)>,
  <EELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1015)>,
  <EELSSpectrum, title: EELS HL SI, dimensions: (140, 140|1015)>])

In [126]:
# Extract the x-axis (energy) values
axes = [ll.axes_manager['Energy loss'].axis for ll in ll_spectrum]

# Find the global minimum and maximum energy values for the intersection
min_energy = max(axis[0] for axis in axes)
max_energy = min(axis[-1] for axis in axes)

# Generate the common axis
common_axis = np.arange(min_energy, max_energy, ll_spectrum[0].axes_manager['Energy loss'].scale)

In [130]:
ll_spectrum[0].data.shape

(140, 140, 1017)

In [154]:
import hyperspy.api as hs
import numpy as np

# Function to interpolate spectrum to the common energy axis
def interpolate_spectrum_to_common_axis(spectrum, common_axis):
    energy_axis = spectrum.axes_manager['Energy loss'].axis
    
    interpolated_data = np.array([
        np.interp(common_axis, energy_axis, spectrum.data[i, j, :])
        for i in range(spectrum.data.shape[0])
        for j in range(spectrum.data.shape[1])
    ]).reshape(spectrum.data.shape[0], spectrum.data.shape[1], len(common_axis))

    # Create a new EELSSpectrum with the interpolated data
    interpolated_spectrum = hs.signals.EELSSpectrum(interpolated_data)

    # Copy the metadata from the original spectrum
    interpolated_spectrum.metadata = spectrum.metadata.deepcopy()

    # Manually copy the original axes properties
    for i,axis in enumerate(spectrum.axes_manager.navigation_axes):
        new_axis = interpolated_spectrum.axes_manager.navigation_axes[i]
        new_axis.scale = axis.scale
        new_axis.units = axis.units
        new_axis.offset = axis.offset
        new_axis.size = axis.size
        new_axis.name = axis.name
        
    # Manually copy the original axes properties
    for i,axis in enumerate(spectrum.axes_manager.signal_axes):
        new_axis = interpolated_spectrum.axes_manager.signal_axes[i]
        new_axis.scale = axis.scale
        new_axis.units = axis.units
        new_axis.offset = axis.offset
        new_axis.size = axis.size
        new_axis.name = axis.name
    


    # Set the new energy axis properties
    energy_axis_index = 0  # Assuming the energy axis is the first signal axis
    interpolated_spectrum.axes_manager.signal_axes[energy_axis_index].scale = common_axis[1] - common_axis[0]
    interpolated_spectrum.axes_manager.signal_axes[energy_axis_index].units = spectrum.axes_manager['Energy loss'].units
    interpolated_spectrum.axes_manager.signal_axes[energy_axis_index].offset = common_axis[0]
    interpolated_spectrum.axes_manager.signal_axes[energy_axis_index].size = len(common_axis)
    interpolated_spectrum.axes_manager.signal_axes[energy_axis_index].axis = common_axis

    return interpolated_spectrum

In [155]:
# Example usage
ll_spectrum_interpolated = [interpolate_spectrum_to_common_axis(ll, common_axis) for ll in ll_spectrum]
hl_spectrum_interpolated = [interpolate_spectrum_to_common_axis(hl, common_axis) for hl in hl_spectrum]

/home/m3learning/anaconda3/envs/northwestern/lib/python3.11/site-packages/hyperspy/signal.py:2451: UserWarning: Setting the `metadata` attribute is deprecated and will be removed in HyperSpy 2.0. Use the `set_item` and `add_dictionary` methods of the `metadata` attribute instead.
  warnings.warn(


In [214]:
ll_spectrum[0].axes_manager

Navigation axis name,size,index,offset,scale,units
x,140,0,-0.0,0.5012668967247009,nm
y,140,0,-0.0,0.5012668967247009,nm
Signal axis name,size,,offset,scale,units
Energy loss,1017,,-23.399419297156285,0.30000001192092896,eV


In [216]:
ll_spectrum_interpolated[2].axes_manager

Navigation axis name,size,index,offset,scale,units
x,140,0,-0.0,0.5012668967247009,nm
y,140,0,-0.0,0.5012668967247009,nm
Signal axis name,size,,offset,scale,units
Energy loss,998,,-20.702908985924964,0.30000001192092896,eV


In [212]:
import matplotlib
from matplotlib.backends.backend_qt5agg import ...
matplotlib.use('Qt5Agg')
matplotlib.rcParams['backend.qt5']='PySide'
# %matplotlib qt
import hyperspy.api as hs
hs.preferences.gui(toolkit="traitsui")

# Plot the spatial image for low loss spectrum and highlight the point (50, 50)
ll_image = ll_spectrum_interpolated[0]
ll_image.plot(navigator='slider',interactive=True)

# # Plot the spatial image for high loss spectrum and highlight the point (50, 50)
# hl_image = hl_spectrum_interpolated[0]
# hl_image.plot(navigator='interactive')


SyntaxError: invalid syntax (392566210.py, line 2)

In [15]:
ll_spectrum[0].axes_manager['Energy loss'].size

1017

In [58]:
[ll.axes_manager['Energy loss'].offset for ll in ll_spectrum]

[-26.213725531435742,
 -24.600000977516174,
 -23.399419297156285,
 -21.302602887305678,
 -20.702908985924964]

In [17]:
[ll.axes_manager['Energy loss'].size for ll in ll_spectrum]

[1017, 1018, 1017, 1015, 1015]

In [19]:
# Determine the global energy range across all spectra
min_energy = min(ll.axes_manager['Energy loss'].offset for ll in ll_spectrum)
max_energy = max(ll.axes_manager['Energy loss'].offset + ll.data.shape[-1] * ll.axes_manager['Energy loss'].scale for ll in ll_spectrum)


In [42]:
axes = [ll.axes_manager['Energy loss'].axis for ll in ll_spectrum]
min_energy = max(axis[0] for axis in axes)
max_energy = min(axis[-1] for axis in axes)
slice_inds = [(np.argwhere(ax==min_energy),np.argwhere(ax==max_energy)) for ax in axes]

In [41]:
min_energy, max_energy

(-20.702908985924964, 278.5862865802281)

In [57]:
import pandas as pd

# Convert the list of axes to a DataFrame for better visualization
axes_df = pd.DataFrame(axes)
display(axes_df)


,0,1,2,3,4,5,6,7,8,9,...,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017
0,-26.213726,-25.913726,-25.613726,-25.313725,-25.013725,-24.713725,-24.413725,-24.113725,-23.813725,-23.513725,...,276.186286,276.486286,276.786287,277.086287,277.386287,277.686287,277.986287,278.286287,278.586287,NaN
1,-24.600001,-24.300001,-24.000001,-23.700001,-23.400001,-23.100001,-22.800001,-22.500001,-22.200001,-21.900001,...,277.800011,278.100011,278.400011,278.700011,279.000011,279.300011,279.600011,279.900011,280.200011,280.500011
2,-23.399419,-23.099419,-22.799419,-22.499419,-22.199419,-21.899419,-21.599419,-21.299419,-20.999419,-20.699419,...,279.000593,279.300593,279.600593,279.900593,280.200593,280.500593,280.800593,281.100593,281.400593,NaN
3,-21.302603,-21.002603,-20.702603,-20.402603,-20.102603,-19.802603,-19.502603,-19.202603,-18.902603,-18.602603,...,281.097409,281.397409,281.697409,281.997409,282.297409,282.597409,282.897409,NaN,NaN,NaN
4,-20.702909,-20.402909,-20.102909,-19.802909,-19.502909,-19.202909,-18.902909,-18.602909,-18.302909,-18.002909,...,281.697103,281.997103,282.297103,282.597103,282.897103,283.197103,283.497103,NaN,NaN,NaN


In [45]:
slice_inds

[(array([], shape=(0, 1), dtype=int64), array([[1016]])),
 (array([], shape=(0, 1), dtype=int64), array([], shape=(0, 1), dtype=int64)),
 (array([], shape=(0, 1), dtype=int64), array([], shape=(0, 1), dtype=int64)),
 (array([], shape=(0, 1), dtype=int64), array([], shape=(0, 1), dtype=int64)),
 (array([[0]]), array([], shape=(0, 1), dtype=int64))]

In [36]:
?ll_spectrum[0].isig

Type:           SpecialSlicersSignal
String form:    <hyperspy.signal.SpecialSlicersSignal object at 0x7f3e7d2de250>
Length:         1017
File:           ~/anaconda3/envs/northwestern/lib/python3.11/site-packages/hyperspy/signal.py
Docstring:      <no docstring>
Init docstring:
Create a slice of the signal. The indexing supports integer,
decimal numbers or strings (containing a decimal number and an units).

>>> s = hs.signals.Signal1D(np.arange(10))
>>> s
<Signal1D, title: , dimensions: (|10)>
>>> s.data
array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
>>> s.axes_manager[0].scale = 0.5
>>> s.axes_manager[0].axis
array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5])
>>> s.isig[0.5:4.].data
array([1, 2, 3, 4, 5, 6, 7])
>>> s.isig[0.5:4].data
array([1, 2, 3])
>>> s.isig[0.5:4:2].data
array([1, 3])
>>> s.axes_manager[0].units = 'µm'
>>> s.isig[:'2000 nm'].data
array([0, 1, 2, 3])

In [28]:
# Determine the global energy range across all spectra
starts = [ll.axes_manager['Energy loss'].offset for ll in ll_spectrum]
ends = [ll.axes_manager['Energy loss'].offset + ll.data.shape[-1] * ll.axes_manager['Energy loss'].scale for ll in ll_spectrum]

In [29]:
starts,ends

([-26.213725531435742,
  -24.600000977516174,
  -23.399419297156285,
  -21.302602887305678,
  -20.702908985924964],
 [278.886286592149,
  280.8000111579895,
  281.7005928264285,
  283.1974092124372,
  283.7971031138179])

In [ ]:

# Define the common energy axis range
common_energy_axis = np.arange(min_energy, max_energy, ll_spectrum[0].axes_manager['Energy loss'].scale)

# Function to resample spectrum to the common energy axis
def resample_spectrum(spectrum, common_axis):
    original_axis = spectrum.axes_manager['Energy loss'].axis
    resampled_data = np.interp(common_axis, original_axis, spectrum.data)
    resampled_spectrum = hs.signals.Signal1D(resampled_data)
    resampled_spectrum.axes_manager[-1].scale = spectrum.axes_manager[-1].scale
    resampled_spectrum.axes_manager[-1].units = spectrum.axes_manager[-1].units
    resampled_spectrum.axes_manager[-1].offset = common_axis[0]
    return resampled_spectrum

# Resample all spectra to the common energy axis
ll_spectrum_resampled = [resample_spectrum(ll, common_energy_axis) for ll in ll_spectrum]
hl_spectrum_resampled = [resample_spectrum(hl, common_energy_axis) for hl in hl_spectrum]

# Optionally, plot the spectra to confirm alignment and resampling
for ll, hl in zip(ll_spectrum_resampled, hl_spectrum_resampled):
    ll.plot(title='Resampled Aligned Low Loss Spectrum')
    hl.plot(title='Resampled Aligned High Loss Spectrum')